## 네이버 금융 핫뉴스 기사 제목을 통한 코스피 주가 상승, 하락 예측

 현재 COVID-19(코로나)로 인한 유래없는 펜데믹 현상으로 인해 전 세계적으로 피해를 보고있으며 경제 또한 이를 피해가지 못하였다. 하지만 위기회라는 신조어가 생긴만큼 떨어진 경제지표에 많은 개인 투자자들이 주식시장에 뛰어들었다. 현재 동학개미운동이라고 불릴만큼 많은 개인투자자들이 새로운 증권 계좌 개설을 하고있으며 금융지식에 대한 뜨거운 관심이 모이고있다. 이에 네이버 금융의 핫뉴스 기사제목을 스크래핑하여 코스피 등락을 예측하는데 활용해보면 어떨까라는 생각이 들었고 전 날의 기사 제목으로 다음 날 코스피 지수 등락을 예측하는 모델을 만들어보았다.

#### 스크래핑과 데이터처리, 한글 형태소 분석에 필요한 패키지들을 불러온다.

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [128]:
#import socket, sys, json
from konlpy.tag import Kkma
from konlpy.utils import pprint

In [235]:
from re import sub

#### 네이버 금융의 핫 뉴스 제목들을 스크래핑하며 날짜 리스트를 주면 해당 날짜들의 핫뉴스 제목들을 데이터프레임으로 반환한다.

In [236]:
def get_hotnews(date_list):
    news_dict = {}
    for d in date_list :
        n_list = []
        for pn in range(1,3):
            url = 'https://finance.naver.com/news/news_list.nhn?mode=RANK&date={}&page={}'.format(d, pn)
            web = urlopen(url)
            source = BeautifulSoup(web,'html.parser',from_encoding='utf-8')
            news_list = source.findAll('li' , {'class':'block1 clearfix'})

            for news in news_list :
                n = news.findAll('a')
                for i in n :
                    title = i.get_text().strip() # 특수문자, 공백 제외 후 의미있는 형태소만 추출
                    title = sub(pattern=r"\d", repl=r"", string=title) # 숫자를 제외
                    title = [i for i in kkma.nouns(title) if len(i) > 1] # 한글자 이상의 단어만 사용
                    n_list.append(title)

            news_dict[d] = n_list
    news_df = pd.DataFrame(news_dict)
    return news_df

#### 2019년 5월 2일부터 2020년 4월 29일까지 휴일, 폐장일을 제외한 데이터를 스크래핑한다. 

In [237]:
dt_index = pd.date_range(start='20190502', end='20200429',freq='B')
dt_list = dt_index.strftime("%Y%m%d").tolist()

#### 휴일, 폐장일을 지정해 목록에서 제외한다.

In [238]:
holiday_list = ['20190506','20190606','20190815','20190912','20190913','20191003','20191009','20191225','20200101','20200124','20200127','20200415','20191231']
for i in holiday_list :
    dt_list.remove(i)

In [239]:
df = get_hotnews(dt_list)

#### 자료출처 : 인베스팅닷컴(https://kr.investing.com/indices/kospi-historical-data)

In [294]:
target = pd.read_csv('kospi.csv')

#### 지수가 떨어진날은 0, 오른날은 1로 타겟을 설정한다.

In [295]:
target_list = [1 if i > 0 else 0 for i in target.values] # up = 1 & down = 0

#### 스크래핑이 끝난 후 하루에 100단어씩 모두 저장한다.

In [296]:
tmp_dict = {date : list(set([word for word_list in df[date] for word in word_list]))[:100] for date in df.columns}

In [331]:
kospi_df = pd.DataFrame(tmp_dict)

#### 더미화(One-hot encoding)를 위해 단어 목록을 정리한다.

In [334]:
col_names = [col[9:] for col in pd.get_dummies(kospi_df).columns]

In [337]:
kospi_df = kospi_df.T

In [338]:
kospi_df.to_csv('kospi_df.csv')

In [339]:
kospi_df.shape

(247, 100)

In [340]:
kospi_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
20190502,회사,동결,일가,진출,케이엠제약,총수,이스트,특징,환율,아이에스커머스,...,이주,일대,솔루션즈,증권사,신입,빌리티,한세,한세예스문화재단,뉴욕,마감
20190503,나스닥,디아이,증자,도시,진출,이벤트,첫날,성수기,반려,요금인하,...,지주,라인,대표번호,공헌,심리,활동,개발,호실적,투자,동물
20190507,선생님,회사,다우,유가,중공업,이벤트,첫날,엑스포,반려,롯데케미칼,...,마감,텔레콤,외환,트럼프,개시,감소세,채용,나흘간,롯데마트,공모전
20190508,회사,주가약세,유가,설문조사,친화,조사,무역전쟁,d,신한금투,위지윅스튜디오,...,미어,요인,점화,뉴욕,하방리스크,마감,마켓,이상,이글루,외환
20190509,우선주,증자,금융당국,급락,청소년,관련주,재평가,특징,내화,해답,...,오산공장,수도권,이미지센서,조선내화,딩스,시황,원자,매출,올해,코스피


#### 매일 체크하여 해당 단어가 있으면 1, 없으면 0으로 인코딩한다.

In [341]:
final_dict = {date : [1 if i in kospi_df.iloc[idx].values else 0 for i in col_names] for idx, date in enumerate(kospi_df.index)}

In [369]:
kospi_df = pd.DataFrame(final_dict)

In [370]:
kospi_df = kospi_df.T 

In [371]:
kospi_df.columns = col_names

In [373]:
kospi_df.drop(kospi_df.tail(1).index,inplace=True) 
# 다음날 예측을 위해 마지막 하루를 제외

In [374]:
kospi_df.shape

(246, 24700)

In [375]:
kospi_df['target'] = target_list[1:] # 다음날 예측을 위해 지수 첫 날 하루를 제외

In [376]:
kospi_df.to_csv('kospi(dummies).csv')

#### 최종적으로 모든 전처리를 마친 데이터는 아래와 같다.

In [377]:
kospi_df.head()

,개발,거래량,경신,공공,금련,금리동결,금리인하,기업,뉴욕,뉴욕증시,...,투자,특징,판매경로,팔고,패러다임,해양,호실적,호텔,흐름,target
20190502,1,1,1,1,1,1,1,1,1,1,...,1,1,0,0,0,0,0,1,0,1
20190503,1,1,1,0,0,0,0,1,0,1,...,1,1,0,0,0,0,1,1,0,1
20190507,0,0,1,0,0,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
20190508,0,0,0,0,0,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,1
20190509,0,1,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,1,0,0,1


In [378]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
import numpy as np

#### 재현성을 위해 시드를 고정하였고, Train, Test 데이터로 나눠준다.

In [379]:
RANDOM_STATE=11
np.random.seed(RANDOM_STATE)

In [380]:
X_train, X_test, y_train, y_test = train_test_split(kospi_df.drop(['target'],axis=1),kospi_df.target,random_state=RANDOM_STATE)

#### Validation data와 Grid search를 사용하여 모델 optimization을 할 것이며, 첫 모델은 앙상블 모델인 Adaboost 를 사용해보았다.

In [381]:
kf = StratifiedKFold(n_splits=10)

In [384]:
params={'n_estimators': [50,100,150], 'learning_rate':[0.01,0.1]}

In [385]:
abc=GridSearchCV(AdaBoostClassifier(), params, cv=kf)
abc.fit(X_train,y_train)
pred=abc.predict(X_test)

C:\Users\taejin\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


#### 아래와 같은 하이퍼파라미터에서 가장 좋은 성능을 나타냈고 55%정도의 정확도를 가졌다. 

In [387]:
abc.best_params_

{'learning_rate': 0.01, 'n_estimators': 100}

In [386]:
abc.score(X_test, y_test)

0.5483870967741935

#### 다음은 역시 앙상블 모델 중 하나인 RandomForest 모델을 사용하여 다음 날 주가 예측을 진행해보았다.

In [391]:
params={'max_depth': [5,7,10,15,None], 'n_estimators': [50,100,150]}

In [392]:
rfc=GridSearchCV(RandomForestClassifier(), params, cv=kf)
rfc.fit(X_train,y_train)
pred=rfc.predict(X_test)

C:\Users\taejin\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


#### 아래 하이퍼파라미터에서 성능이 가장 뛰어났으며 Adaboost 보다 높은 56.5%의 성능을 기록하였다.

In [388]:
rfc.best_params_

{'max_depth': 10, 'n_estimators': 50}

In [389]:
rfc.score(X_test,y_test)

0.5645161290322581

### Review 
 간단 스크래핑을 통한 주가 예측이였지만 하루 단위를 56.5% 정도로 예측할 수 있었다. 주말이나 긴 연휴 등 이틀 이상의 긴 폐장일을 제외하면 성능은 60% 더 이상도 쉽게 올라갈 수 있을 것으로 보인다. 이는 즉 간단한 스크래핑을 통해 만든 이 모델으로도 지수에 투자한다면 쉽게 반 이상의 확률로 성공할 수 있다는 것을 의미하며 좋은 하드웨어 환경에서 더 많은 데이터나 단어를 사용한다면 연구할만한 가치가 있을 것으로 예상된다. 